In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
%matplotlib inline


In [205]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
home_directory = os.path.expanduser("~")
omnetpp_directory = os.path.join(home_directory, "omnetpp-6.0.2")
vec_directory = os.path.join(omnetpp_directory, "samples", "thesis", "results")

In [206]:
dispatchingAlgos = ["Random", "Greedy", "Proposed"] # ["Random", "Greedy", "Proposed"]
schedulingAlgos = ["FIFO", "FIFO", "Proposed"] # ["Random", "Greedy", "Proposed"]
numUsers = [user for user in range(500, 501, 100)]
repetition = 1
for idx, dispatchingAlgo in enumerate(dispatchingAlgos):
    for numUser in numUsers:
        for i in range(repetition):
            command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}-vec.csv {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}.vec"
            command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}-sca.csv {vec_directory}/General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}.sca"
            os.system(command_vec)
            os.system(command_sca)


Exported 125 vectors
Exported 5 scalars, 6510 parameters
Exported 125 vectors
Exported 5 scalars, 6510 parameters
Exported 125 vectors
Exported 5 scalars, 6510 parameters


In [219]:

algoSuccessRate = []

for idx, dispatchingAlgo in enumerate(dispatchingAlgos):
    successRate = []
    for numUser in numUsers:
        total = 0
        for i in range(repetition):
            sca_out = pd.read_csv(f"General-numUsers={numUser},dispatchingAlgo={dispatchingAlgo},schedulingAlgo={schedulingAlgos[idx]}-#{i}-sca.csv")
            print(f"{dispatchingAlgo}-{numUser}-{i}")
            display(sca_out[(sca_out['type'] == 'scalar')])
            # sca_mean_row = sca_out[(sca_out['type'] == 'scalar') & (sca_out['name'] == 'isCompleted:mean')]
            sca_mean_row = sca_out[(sca_out['type'] == 'scalar')]
            total += float(sca_mean_row[sca_mean_row['name'] == 'isCompleted:mean'].iloc[0]['value'])
        avg = total / repetition
        successRate.append(avg)
    algoSuccessRate.append(successRate)

for idx, dispatchingAlgo in enumerate(dispatchingAlgos):
    print(f"{dispatchingAlgo} successRate is {algoSuccessRate[idx]}")


Random-500-0


,run,type,module,name,attrname,attrvalue,value
40,General-0-20240218-05:40:11-117174,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,1570356
44,General-0-20240218-05:40:11-117174,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,0
48,General-0-20240218-05:40:11-117174,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,1570356
52,General-0-20240218-05:40:11-117174,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,954209
56,General-0-20240218-05:40:11-117174,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.60763865008953


Greedy-500-0


/tmp/ipykernel_69770/320236065.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total += float(sca_mean_row[sca_out['name'] == 'isCompleted:mean'].iloc[0]['value'])


,run,type,module,name,attrname,attrvalue,value
40,General-1-20240218-05:40:11-117175,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,1570736
44,General-1-20240218-05:40:11-117175,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,0
48,General-1-20240218-05:40:11-117175,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,1570736
52,General-1-20240218-05:40:11-117175,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,1384947
56,General-1-20240218-05:40:11-117175,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.88171850648358


Proposed-500-0


/tmp/ipykernel_69770/320236065.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total += float(sca_mean_row[sca_out['name'] == 'isCompleted:mean'].iloc[0]['value'])


,run,type,module,name,attrname,attrvalue,value
40,General-2-20240218-05:40:30-117197,scalar,EdgeEnv.finishedTaskCollector,partialComplete:count,NaN,NaN,1570730
44,General-2-20240218-05:40:30-117197,scalar,EdgeEnv.finishedTaskCollector,partialComplete:sum,NaN,NaN,206459
48,General-2-20240218-05:40:30-117197,scalar,EdgeEnv.finishedTaskCollector,isCompleted:count,NaN,NaN,1570730
52,General-2-20240218-05:40:30-117197,scalar,EdgeEnv.finishedTaskCollector,isCompleted:sum,NaN,NaN,1319510
56,General-2-20240218-05:40:30-117197,scalar,EdgeEnv.finishedTaskCollector,isCompleted:mean,NaN,NaN,0.84006162739618


Random successRate is [0.60763865008953]
Greedy successRate is [0.88171850648358]
Proposed successRate is [0.84006162739618]


/tmp/ipykernel_69770/320236065.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total += float(sca_mean_row[sca_out['name'] == 'isCompleted:mean'].iloc[0]['value'])


In [208]:
proposed_500_vec_out = pd.read_csv(f"General-numUsers=500,dispatchingAlgo=Proposed,schedulingAlgo=Proposed-#0-vec.csv")
proposed_500_vec_out = proposed_500_vec_out[proposed_500_vec_out['type'] == 'vector']
tmp = proposed_500_vec_out['vecvalue'].str.split(expand=True)
tmp = tmp.transpose()
proposed_500_vec_out['name']
# print([name for name in proposed_500_vec_out['name'].to_list()])


40          memoryLoading:vector
44             CPULoading:vector
48      totalTaskFinished:vector
52     totalTaskCompleted:vector
56          totalTaskDrop:vector
                 ...            
520         memoryLoading:vector
524            CPULoading:vector
528     totalTaskFinished:vector
532    totalTaskCompleted:vector
536         totalTaskDrop:vector
Name: name, Length: 125, dtype: object

In [209]:
display(tmp)

,40,44,48,52,56,60,64,68,72,76,...,500,504,508,512,516,520,524,528,532,536
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.1,0.05,199,199,0,0.08974206348,0.0448710318,534,525,0,...,0,0,253,242,344,0.01,0.005,186,183,0
2,0.1,0.05,200,200,0,0.08672619042,0.0433630948,557,550,0,...,0.02,0.01,220,217,166,0.008,0.004,146,143,0
3,0.1,0.05,200,200,0,0.08998809516,0.0449940474,578,570,0,...,0.02474999998,0.0123749998,251,243,233,0.0075,0.00375,180,177,0
4,0.1,0.05,200,200,0,0.08945634918,0.0447281748,563,552,0,...,0.01833333332,0.0091666666,240,222,288,0.01,0.005,145,138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0.1,0.05,200,200,0,0.086547619,0.0432738096,555,548,0,...,0.03019047616,0.015095238,254,243,188,0.01,0.005,172,168,0
297,0.1,0.05,200,200,0,0.08969841276,0.044849207,569,550,0,...,0.02402777776,0.0120138888,273,260,222,0.00625,0.003125,186,172,0
298,0.1,0.05,200,200,0,0.08766269842,0.0438313494,557,540,0,...,0.011,0.0055,233,226,176,0.00666666666,0.0033333332,151,148,0
299,0.1,0.05,200,200,0,0.08741666666,0.0437083332,567,557,0,...,0.01,0.005,261,245,200,0,0,178,169,0


In [210]:
# random_command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/random_vec.csv {vec_directory}/Random-#0.vec"
# random_command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/random_sca.csv {vec_directory}/Random-#0.sca"

# os.system(random_command_vec)
# os.system(random_command_sca)
# random_out = pd.read_csv('random_vec.csv')
# display(random_out.head())
# print(random_out['type'].unique())

In [211]:
# random_vector_type_df = random_out[random_out['type'] == 'vector']
# display(random_vector_type_df)
# random_vecval_df = random_vector_type_df['vecvalue'].str.split(expand=True)
# display(random_vecval_df)
# random_tasks_df = random_vecval_df.transpose()
# random_tasks_df.columns = [name.split(':')[0] for name in random_vector_type_df['name'].to_list()]
# # display(random_tasks_df)
# random_tasks_numeric_df = random_tasks_df.apply(pd.to_numeric, errors='ignore')
# display(random_tasks_numeric_df)

In [212]:
# greedy_command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/greedy_vec.csv {vec_directory}/Greedy-#0.vec"
# greedy_command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/greedy_sca.csv {vec_directory}/Greedy-#0.sca"

# os.system(greedy_command_vec)
# os.system(greedy_command_sca)

# greedy_out = pd.read_csv('greedy_vec.csv')
# display(greedy_out.head())

In [213]:
# greedy_vector_type_df = greedy_out[greedy_out['type'] == 'vector']
# display(greedy_vector_type_df)
# greedy_vecval_df = greedy_vector_type_df['vecvalue'].str.split(expand=True)
# display(greedy_vecval_df)
# greedy_tasks_df = greedy_vecval_df.transpose()
# greedy_tasks_df.columns = [name.split(':')[0] for name in greedy_vector_type_df['name'].to_list()]
# # display(greedy_tasks_df)
# greedy_tasks_numeric_df = greedy_tasks_df.apply(pd.to_numeric, errors='ignore')
# display(greedy_tasks_numeric_df)


In [214]:
# proposed_command_vec = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/proposed_vec.csv {vec_directory}/Proposed-#0.vec"
# proposed_command_sca = f"{omnetpp_directory}/bin/opp_scavetool export -F CSV-R -o {vec_directory}/proposed_sca.csv {vec_directory}/Proposed-#0.sca"

# os.system(proposed_command_vec)
# os.system(proposed_command_sca)

# proposed_out = pd.read_csv('proposed_vec.csv')
# display(proposed_out.head())

In [215]:
# proposed_vector_type_df = proposed_out[proposed_out['type'] == 'vector']
# display(proposed_vector_type_df)
# proposed_vecvalue_column_df = proposed_vector_type_df[['vecvalue']]

In [216]:
# proposed_vecval_df = proposed_vecvalue_column_df['vecvalue'].str.split(expand=True)
# display(proposed_vecval_df)

In [217]:
# proposed_tasks_df = proposed_vecval_df.transpose()
# proposed_tasks_df.columns = [name.split(':')[0] for name in proposed_vector_type_df['name'].to_list()]
# # display(proposed_tasks_df)
# proposed_tasks_numeric_df = proposed_tasks_df.apply(pd.to_numeric, errors='ignore')
# display(proposed_tasks_numeric_df)

In [218]:
# print(random_tasks_numeric_df[random_tasks_numeric_df['isCompleted'] == 1].size / random_tasks_numeric_df.size)
# print(greedy_tasks_numeric_df[greedy_tasks_numeric_df['isCompleted'] == 1].size / greedy_tasks_numeric_df.size)
# print(proposed_tasks_numeric_df[proposed_tasks_numeric_df['isCompleted'] == 1].size / proposed_tasks_numeric_df.size)